<a href="https://colab.research.google.com/github/ljqljqljq8/0115_Git_Build/blob/main/Type_One.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Label 解析 / 不同类型样本示例**

In [ ]:
import re
def parse_label(label):
    faults = []
    # 提取故障信息：故障N：类型，Bounding Box...
    fault_matches = re.findall(r"故障(\d+)：([^，]+)，Bounding Box([^；]+)", label)
    for match in fault_matches:
        fault_num, fault_type, bbox_desc = match
        # 提取红外和可见光坐标
        ir_coords = re.search(r"在红外图像的坐标为\(([^)]+)\), \(([^)]+)\)", bbox_desc)
        vis_coords = re.search(r"在可见光图像的坐标为\(([^)]+)\), \(([^)]+)\)", bbox_desc)

        ir_coord_str = f"坐标({ir_coords.group(1)})到({ir_coords.group(2)})" if ir_coords else None
        vis_coord_str = f"坐标({vis_coords.group(1)})到({vis_coords.group(2)})" if vis_coords else None

        # 处理“只在红外图像出现”的情况
        if "只在红外图像出现" in bbox_desc:
            ir_coords = re.search(r"只在红外图像出现，坐标为\(([^)]+)\), \(([^)]+)\)", bbox_desc)
            ir_coord_str = f"坐标({ir_coords.group(1)})到({ir_coords.group(2)})" if ir_coords else None
            vis_coord_str = None

        faults.append({
            "type": fault_type.strip(),
            "ir_coords": ir_coord_str,
            "vis_coords": vis_coord_str
        })

    return faults

In [ ]:
# 单个单种缺陷
label1 = (
    "总共有1个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
)

# 多种不同类型缺陷（草木遮挡+边缘积尘+组串开路）
label2 = (
    "总共有3个故障；故障1：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障2：边缘积尘，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障3：组串开路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)

# 多种缺陷（表面破损+多处草木遮挡+边缘积尘+内部缺陷）
label3 = (
    "总共有5个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：草木遮挡，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障4：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)；"
    "故障5：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)

label4 = (
    "没有缺陷"
)


### **不同缺陷定义**

In [ ]:
descriptions = {
    "组串短路": {
        "ir_feature": "连续点状热斑",
        "vis_feature": "无明显特征",
        "impact": "会导致光伏面板部分或全部停发，因线路过热形成严重热斑，存在设备烧毁或火灾的潜在风险",
        "peocess priority": 1,
        "reason": "安全风险最高，需要立即处理避免火灾发生",
        "repair_strategy": "需要通过断开汇流箱熔断器断开短路组串回路，排查并修复短路故障点，然后使用绝缘电阻测试仪验证无短路并通过红外热像仪确认连续点状热斑消失，通常无需更换组件。",
        "cause": "常见原因是线路绝缘层破损（如线缆磨损、接头进水）、接线错误（正负极接反）、端子松动短路"
    },
    "组串开路": {
        "ir_feature": "整体热斑",
        "vis_feature": "无明显特征",
        "impact": "会直接导致整个组串停止发电，形成大范围热斑，显著降低系统发电效率",
        "peocess priority": 2,
        "reason": "影响范围最大，需立即排查",
        "repair_strategy": "需要检查汇流箱熔断器、组件间连接和插头，修复开路点后使用电压表测试组串连续性并通过红外热像仪确认整体热斑消除，通常无需更换组件。",
        "cause": "主要因线路接头松动/脱落、线缆断裂、接线端子氧化接触不良导致"
    },
    "二极管故障": {
        "ir_feature": "条状热斑",
        "vis_feature": "无明显特征",
        "impact": "因热斑效应导致局部发电效率下降30%以上，存在加速组件老化的潜在风险",
        "peocess priority": 3,
        "reason": "可能加速老化，需尽快处理",
        "repair_strategy": "需要检查接线盒，若二极管可单独更换则进行更换，然后使用多用表测试二极管导通并通过红外热像仪确认条状热斑消除；若二极管不可单独更换或更换成本过高，则需更换组件。",
        "cause": "部分情况属于线路问题，若二极管外部线路存在虚接、过载，可能导致二极管过流烧毁；但内置二极管自身老化故障不属于线路问题。"
    },
    "内部缺陷": {
        "ir_feature": "点状热斑",
        "vis_feature": "无明显特征",
        "impact": "属于隐性故障，会降低发电效率下降10%-50%，而且会持续恶化，存在局部瘫痪的潜在风险",
        "peocess priority": 4,
        "reason": "故障持续恶化，无法外部修复",
        "repair_strategy": "需要使用EL电致发光测试、红外热像仪或I-V曲线分析确认缺陷范围和功率衰减程度，若为电池片隐裂、断栅、焊带脱落等严重问题，则需更换组件。",
        "cause": "如电池片隐裂、断栅、焊带脱落等，通常由于制造缺陷、安装不当或环境因素引起"
    },
    "组件异位": {
        "ir_feature": "大面积区域热斑",
        "vis_feature": "太阳光反射",
        "impact": "因角度偏移导致采光受到影响，可能有10%-30%的发电效率下降，存在结构不稳定的潜在风险",
        "peocess priority": 5,
        "reason": "结构不稳定，有掉落风险，需及时矫正",
        "repair_strategy": "需要重新固定或对齐异位组件，确保安装支架稳固，然后使用红外热像仪检查大面积热斑消失并测试组串电压电流平衡，通常无需更换组件。",
        "cause": "由于安装不稳固、风力或地震等外部力导致角度偏移"
    },
    "表面破损": {
        "ir_feature": "区域热斑",
        "vis_feature": "破损现象",
        "impact": "因破损导致透光率下降，可能导致发电效率下降10%-30%，存在进水损坏电路的潜在风险",
        "peocess priority": 6,
        "reason": "玻璃破碎后会进水、进尘，损坏内部电路，需尽快处理以防止故障扩大。",
        "repair_strategy": "需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。",
        "cause": "由于外部冲击、冰雹、动物啃咬等造成玻璃或背板破损"
    },
    "草木遮挡": {
        "ir_feature": "区域热斑",
        "vis_feature": "草木",
        "impact": "会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率",
        "peocess priority": 7,
        "reason": "遮挡面积会随植物生长扩大，快速降低局部发电效率，需定期清理。",
        "repair_strategy": "需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，通常无需更换组件。",
        "cause": "由于周边植物生长未及时清理导致"
    },
    "固定设施遮挡": {
        "ir_feature": "区域热斑或阴影",
        "vis_feature": "固定设施或阴影",
        "impact": "因固定设施导致光伏板局部面积遮挡，通常引起的发电效率下降小于10%",
        "peocess priority": 8,
        "reason": "遮挡位置固定但持续影响，潜在风险较低，需通过调整设施角度、位置来解决",
        "repair_strategy": "需要评估并移除或调整固定设施如支架或围栏，若无法移除则重新布局组件位置，然后使用红外热像仪验证阴影消除并进行I-V曲线测试确认发电效率恢复，通常无需更换组件。",
        "cause": "由于安装时未考虑固定设施如支架或围栏的位置导致"
    },
    "表面污迹": {
        "ir_feature": "区域热斑",
        "vis_feature": "污渍",
        "impact": "会轻微影响光伏板透光率，通常引起的发电效率下降小于10%",
        "peocess priority": 9,
        "reason": "潜在风险低，若污迹干燥且无腐蚀性，可按周期清理",
        "repair_strategy": "需要使用水或中性清洁剂去除组件表面污渍，然后在辐照度较好的环境下，用红外设备复测组件，如热斑依然存在，使用I-V曲线测试仪检测组件功率，若功率低于期望值，则需更换组件。",
        "cause": "由于泥土、鸟粪、工业污染物等积累导致"
    },
    "边缘积尘": {
        "ir_feature": "区域热斑",
        "vis_feature": "边缘积尘",
        "impact": "因积尘量少，污染程度较低，通常引起的发电效率下降小于10%",
        "peocess priority": 10,
        "reason": "影响相对较小，可在常规清洁时顺带处理",
        "repair_strategy": "需要清除组件边缘积尘，然后通过红外设备复测组件，通常无需更换组件。如现象普遍，应对全站组件边缘进行重点清洗。",
        "cause": "由于灰尘在组件边缘积累，通常因清洁不当或环境尘土多导致"
    },
}

### TypeOne_id3 ——**要是光伏板有隐裂啥的，这两种照片能看出来不？**

In [ ]:
def TypeOne_id3(label):
    # 固定科学说明部分
    science_intro = (
        "光伏板的表面破损在红外图像中通常表现为区域光斑，这是因为破损会导致局部热分布不均，形成明显的温度差异区域；"
        "在可见光图像中，则可以观察到直接的物理损伤迹象，如裂纹、碎片或表面缺损。"
    )
    faults = parse_label(label)
    # 检查是否有表面破损
    damage_faults = [f for f in faults if f["type"] == "表面破损"]

    if damage_faults:
        # 有表面破损：合并坐标描述
        ir_coords_list = [df["ir_coords"] for df in damage_faults]
        vis_coords_list = [df["vis_coords"] for df in damage_faults]
        ir_coords_str = "和".join(ir_coords_list)
        vis_coords_str = "和".join(vis_coords_list)
        damage_part = (
            f"对这组图像进行分析，检测到{len(damage_faults)}处表面破损故障，"
            f"在红外图像{'依次' if len(damage_faults) > 1 else ''}呈现在{ir_coords_str}区域，同时对应在可见光图像中{vis_coords_str}的区域，显示出明显的破损特征。"
        )
        answer = f"{science_intro}{damage_part}"
    else:
        # 无表面破损：说明未检测到
        no_damage_desc = (
            "对这组图像进行分析，在红外图像中没有呈现区域光斑，可见光图像中也没有呈现裂纹或碎片等物理损伤迹象，"
            "所以未检测到表面破损。"
        )
        answer = f"{science_intro}{no_damage_desc}"
    return answer

# 测试用例1：单处表面破损
label1 = (
    "总共有2个故障；故障1：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)；"
    "故障2：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
)
print("测试用例1 输出：")
print(TypeOne_id3(label1))

# 测试用例2：无表面破损
label2 = (
    "总共有3个故障；故障1：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障2：内部缺陷，Bounding Box只在红外图像出现，坐标为(325, 282), (338, 291)；"
    "故障3：内部缺陷，Bounding Box只在红外图像出现，坐标为(532, 24), (621, 38)"
)
print("\n测试用例2 输出：")
print(TypeOne_id3(label2))

# 测试用例3：多处表面破损
label3 = (
    "总共有3个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：表面破损，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障3：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("\n测试用例3 输出：")
print(TypeOne_id3(label3))

测试用例1 输出：
光伏板的表面破损在红外图像中通常表现为区域光斑，这是因为破损会导致局部热分布不均，形成明显的温度差异区域；在可见光图像中，则可以观察到直接的物理损伤迹象，如裂纹、碎片或表面缺损。对这组图像进行分析，检测到1处表面破损故障，在红外图像呈现在坐标(365, 148)到(421, 176)区域，同时对应在可见光图像中坐标(1399, 617)到(1598, 725)的区域，显示出明显的破损特征。

测试用例2 输出：
光伏板的表面破损在红外图像中通常表现为区域光斑，这是因为破损会导致局部热分布不均，形成明显的温度差异区域；在可见光图像中，则可以观察到直接的物理损伤迹象，如裂纹、碎片或表面缺损。对这组图像进行分析，在红外图像中没有呈现区域光斑，可见光图像中也没有呈现裂纹或碎片等物理损伤迹象，所以未检测到表面破损。

测试用例3 输出：
光伏板的表面破损在红外图像中通常表现为区域光斑，这是因为破损会导致局部热分布不均，形成明显的温度差异区域；在可见光图像中，则可以观察到直接的物理损伤迹象，如裂纹、碎片或表面缺损。对这组图像进行分析，检测到2处表面破损故障，在红外图像依次呈现在坐标(365, 148)到(421, 176)和坐标(200, 300)到(250, 350)区域，同时对应在可见光图像中坐标(1399, 617)到(1598, 725)和坐标(800, 900)到(850, 950)的区域，显示出明显的破损特征。


### TypeOne_id6 ——**发电系统会瘫痪吗？**

In [ ]:
def TypeOne_id6(label):

    # 缺陷类型对应的维修措施
    repair_strategies = {
        "草木遮挡": "需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，无需更换组件。",
        "固定设施遮挡": "需要评估并移除或调整固定设施如支架或围栏，若无法移除则重新布局组件位置，然后使用红外热像仪验证阴影消除并进行I-V曲线测试确认发电效率恢复，无需更换组件。",
        "表面污迹": "需要使用水或中性清洁剂去除组件表面污渍，然后在辐照度较好的环境下，用红外设备复测组件，如热斑依然存在，使用I-V曲线测试仪检测组件功率，若功率低于期望值，则需更换组件。",
        "表面破损": "需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。",
        "内部缺陷": "需要使用EL电致发光测试、红外热像仪或I-V曲线分析确认缺陷范围和功率衰减程度，若为电池片隐裂、断栅、焊带脱落等严重问题，则需更换组件。",
        "边缘积尘": "需要清除组件边缘积尘，然后通过红外设备复测组件，无需更换组件。如现象普遍，应对全站组件边缘进行重点清洗。",
        "组件异位": "需要重新固定或对齐异位组件，确保安装支架稳固，然后使用红外热像仪检查大面积热斑消失并测试组串电压电流平衡，无需更换组件。",
        "二极管故障": "需要检查接线盒，若二极管可单独更换则进行更换，然后使用多用表测试二极管导通并通过红外热像仪确认条状热斑消除；若二极管不可单独更换或更换成本过高，则需更换组件。",
        "组串短路": "需要通过断开汇流箱熔断器断开短路组串回路，排查并修复短路故障点，然后使用绝缘电阻测试仪验证无短路并通过红外热像仪确认连续点状热斑消失，无需更换组件。",
        "组串开路": "需要检查汇流箱熔断器、组件间连接和插头，修复开路点后使用电压表测试组串连续性并通过红外热像仪确认整体热斑消除，无需更换组件。"
    }

    faults = parse_label(label)
    print(faults)
    # 检测到的缺陷类型
    fault_types = list(set(f["type"] for f in faults))  # 去重，获取唯一缺陷类型
    if not fault_types:
        return "对这组图像进行分析，未检测到任何缺陷，无需更换组件或采取维修措施。"

    # 分离高严重性（可能需换板）和低严重性（无需换板）缺陷
    high_severity = ["表面破损", "内部缺陷", "二极管故障"]
    high_severity_faults = [f for f in faults if f["type"] in high_severity]
    low_severity_faults = [f for f in faults if f["type"] not in high_severity]

    # 总述
    answer_parts = []
    answer_parts.append(f"对这组图像进行分析，检测到{len(fault_types)}种缺陷：{'、'.join(fault_types)}。")

    # 处理高严重性缺陷（可能需换板）
    replace_needed = []
    for fault in high_severity_faults:
        fault_type = fault["type"]
        repair = repair_strategies[fault_type]
        # 优先使用可见光坐标，若无则用红外坐标
        coords = fault["vis_coords"] or fault["ir_coords"] or "指定区域"
        coord_type = "可见光" if fault["vis_coords"] else "热成像"
        replace_needed.append(f"在{coord_type}{coords}的范围检测到{fault_type}，{repair}")

    # 处理低严重性缺陷（无需换板），按类型合并
    non_replace_needed = []
    low_severity_types = list(set(f["type"] for f in low_severity_faults))
    if low_severity_types:
        low_severity_desc = []
        for fault_type in low_severity_types:
            # 获取该类型的所有坐标，优先使用可见光坐标
            coords_list = []
            for f in low_severity_faults:
                if f["type"] == fault_type:
                    coord = f["vis_coords"] or f["ir_coords"] or "指定区域"
                    coord_type = "可见光" if f["vis_coords"] else "热成像"
                    coords_list.append(f"{coord_type}{coord}")
            coords_str = "和".join(coords_list)
            repair = repair_strategies[fault_type]
            low_severity_desc.append(f"{fault_type}，出现在在{coords_str}的范围，{repair}")
        non_replace_needed.append(f"{' '.join(low_severity_desc)}")

    # 综合描述
    if replace_needed:
        answer_parts.append("需采取以下维修措施：")
        answer_parts.extend(replace_needed)
        if non_replace_needed:
            answer_parts.append("此外，还需处理以下一般情况下无需更换组件的缺陷：")
            answer_parts.extend(non_replace_needed)
    else:
        answer_parts.append("需采取以下维修措施，通常无需更换组件：")
        answer_parts.extend(non_replace_needed if non_replace_needed else ["无需采取额外维修措施。"])

    return " ".join(answer_parts)

# 测试用例1：单种高严重性缺陷（表面破损）
label1 = (
    "总共有1个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
)
print("测试用例1 输出：")
print(TypeOne_id6(label1))

# 测试用例2：多种不同类型低严重性缺陷（草木遮挡+边缘积尘+组串开路）
label2 = (
    "总共有3个故障；故障1：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障2：边缘积尘，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障3：组串开路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("\n测试用例2 输出：")
print(TypeOne_id6(label2))

# 测试用例3：多种缺陷（表面破损+多处草木遮挡+边缘积尘+内部缺陷）
label3 = (
    "总共有5个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：草木遮挡，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障4：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)；"
    "故障5：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("\n测试用例3 输出：")
print(TypeOne_id6(label3))

label4 = (
    "没有缺陷"
)
print("测试用例4 输出：")
print(TypeOne_id6(label4))

测试用例1 输出：
[{'type': '表面破损', 'ir_coords': '坐标(365, 148)到(421, 176)', 'vis_coords': '坐标(1399, 617)到(1598, 725)'}]
对这组图像进行分析，检测到1种缺陷：表面破损。 需采取以下维修措施： 在可见光坐标(1399, 617)到(1598, 725)的范围检测到表面破损，需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。

测试用例2 输出：
[{'type': '草木遮挡', 'ir_coords': '坐标(73, 81)到(110, 108)', 'vis_coords': '坐标(259, 345)到(289, 398)'}, {'type': '边缘积尘', 'ir_coords': '坐标(200, 300)到(250, 350)', 'vis_coords': '坐标(800, 900)到(850, 950)'}, {'type': '组串开路', 'ir_coords': '坐标(557, 155)到(587, 165)', 'vis_coords': None}]
对这组图像进行分析，检测到3种缺陷：草木遮挡、边缘积尘、组串开路。 需采取以下维修措施，通常无需更换组件： 草木遮挡，出现在在可见光坐标(259, 345)到(289, 398)的范围，需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，无需更换组件。 边缘积尘，出现在在可见光坐标(800, 900)到(850, 950)的范围，需要清除组件边缘积尘，然后通过红外设备复测组件，无需更换组件。如现象普遍，应对全站组件边缘进行重点清洗。 组串开路，出现在在热成像坐标(557, 155)到(587, 165)的范围，需要检查汇流箱熔断器、组件间连接和插头，修复开路点后使用电压表测试组串连续性并通过红外热像仪确认整体热斑消除，无需更换组件。

测试用例3 输出：
[{'type': '表面破损', 'ir_coords': '坐标(365, 148)到(421, 176)', 'vis_coor

### TypeOne_id8 ——**这片板子是修理还是直接更换？**

In [ ]:
def TypeOne_id8(label):
    # 缺陷类型对应的维修措施
    repair_strategies = {
        "草木遮挡": "需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，无需更换组件。",
        "固定设施遮挡": "需要评估并移除或调整固定设施如支架或围栏，若无法移除则重新布局组件位置，然后使用红外热像仪验证阴影消除并进行I-V曲线测试确认发电效率恢复，无需更换组件。",
        "表面污迹": "需要使用水或中性清洁剂去除组件表面污渍，然后在辐照度较好的环境下，用红外设备复测组件，如热斑依然存在，使用I-V曲线测试仪检测组件功率，若功率低于期望值，则需更换组件。",
        "表面破损": "需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。",
        "内部缺陷": "需要使用EL电致发光测试、红外热像仪或I-V曲线分析确认缺陷范围和功率衰减程度，若为电池片隐裂、断栅、焊带脱落等严重问题，则需更换组件。",
        "边缘积尘": "需要清除组件边缘积尘，然后通过红外设备复测组件，无需更换组件。如现象普遍，应对全站组件边缘进行重点清洗。",
        "组件异位": "需要重新固定或对齐异位组件，确保安装支架稳固，然后使用红外热像仪检查大面积热斑消失并测试组串电压电流平衡，无需更换组件。",
        "二极管故障": "需要检查接线盒，若二极管可单独更换则进行更换，然后使用多用表测试二极管导通并通过红外热像仪确认条状热斑消除；若二极管不可单独更换或更换成本过高，则需更换组件。",
        "组串短路": "需要通过断开汇流箱熔断器断开短路组串回路，排查并修复短路故障点，然后使用绝缘电阻测试仪验证无短路并通过红外热像仪确认连续点状热斑消失，无需更换组件。",
        "组串开路": "需要检查汇流箱熔断器、组件间连接和插头，修复开路点后使用电压表测试组串连续性并通过红外热像仪确认整体热斑消除，无需更换组件。"
    }

    faults = parse_label(label)

    # 检测到的缺陷类型
    fault_types = list(set(f["type"] for f in faults))
    if not fault_types:
        return "对这组图像进行分析，未检测到有任何缺陷，无需修理或更换组件。"

    # 分离高严重性（可能需换板）和低严重性（无需换板）缺陷
    high_severity = ["表面破损", "内部缺陷", "二极管故障"]
    high_severity_faults = [f for f in faults if f["type"] in high_severity]
    low_severity_faults = [f for f in faults if f["type"] not in high_severity]

    # 生成回答
    answer_parts = []
    answer_parts.append(f"对这组图像进行分析，检测到{len(fault_types)}种缺陷：{'、'.join(fault_types)}。")

    # 处理高严重性缺陷（可能需换板）
    replace_needed = []
    for fault in high_severity_faults:
        fault_type = fault["type"]
        repair = repair_strategies[fault_type]
        coords = fault["vis_coords"] or fault["ir_coords"] or "指定区域"
        coord_type = "可见光" if fault["vis_coords"] else "热成像"
        if fault_type == "表面破损":
            replace_needed.append(f"在{coord_type}{coords}的范围检测到{fault_type}，{repair}")
        elif fault_type == "内部缺陷":
            replace_needed.append(f"在{coord_type}{coords}的范围检测到{fault_type}，需评估电池片隐裂或断栅程度，若严重则需更换组件。")
        elif fault_type == "二极管故障":
            replace_needed.append(f"在{coord_type}{coords}的范围检测到{fault_type}，需检查接线盒，若二极管不可单独更换则需更换组件，否则可通过更换二极管修复。")

    # 处理低严重性缺陷（无需换板），按类型合并
    non_replace_needed = []
    low_severity_types = list(set(f["type"] for f in low_severity_faults))
    if low_severity_types:
        low_severity_desc = []
        for fault_type in low_severity_types:
            repair = repair_strategies[fault_type]
            coords_list = []
            for f in low_severity_faults:
                if f["type"] == fault_type:
                    coord = f["vis_coords"] or f["ir_coords"] or "指定区域"
                    coord_type = "可见光" if f["vis_coords"] else "热成像"
                    coords_list.append(f"{coord_type}{coord}")
            coords_str = "和".join(coords_list)
            low_severity_desc.append(f"对于{fault_type}，在{coords_str}的范围，{repair}")
        non_replace_needed.append(f"{' '.join(low_severity_desc)}")

    # 综合描述
    if replace_needed:
        answer_parts.append("需评估是否更换组件：")
        answer_parts.extend(replace_needed)
        if non_replace_needed:
            answer_parts.append("其他缺陷通常无需更换组件，只需修理：")
            answer_parts.extend(non_replace_needed)
    else:
        answer_parts.append("一般情况下无需更换组件，只需修理：")
        answer_parts.extend(non_replace_needed if non_replace_needed else ["无需采取额外维修措施。"])

    return " ".join(answer_parts)

# 测试用例1：单种高严重性缺陷（表面破损）
label1 = (
    "总共有1个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
)
print("测试用例1 输出：")
print(TypeOne_id8(label1))

# 测试用例2：多种不同类型低严重性缺陷（草木遮挡+边缘积尘+组串开路）
label2 = (
    "总共有3个故障；故障1：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障2：边缘积尘，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障3：组串开路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("\n测试用例2 输出：")
print(TypeOne_id8(label2))

# 测试用例3：多种缺陷（表面破损+多处草木遮挡+边缘积尘+内部缺陷）
label3 = (
    "总共有5个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：草木遮挡，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障4：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)；"
    "故障5：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("\n测试用例3 输出：")
print(TypeOne_id8(label3))

label4 = (
    "没有缺陷"
)
print("测试用例4 输出：")
print(TypeOne_id8(label4))

测试用例1 输出：
对这组图像进行分析，检测到1种缺陷：表面破损。 需评估是否更换组件： 在可见光坐标(1399, 617)到(1598, 725)的范围检测到表面破损，需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。

测试用例2 输出：
对这组图像进行分析，检测到3种缺陷：草木遮挡、边缘积尘、组串开路。 一般情况下无需更换组件，只需修理： 对于草木遮挡，在可见光坐标(259, 345)到(289, 398)的范围，需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，无需更换组件。 对于边缘积尘，在可见光坐标(800, 900)到(850, 950)的范围，需要清除组件边缘积尘，然后通过红外设备复测组件，无需更换组件。如现象普遍，应对全站组件边缘进行重点清洗。 对于组串开路，在热成像坐标(557, 155)到(587, 165)的范围，需要检查汇流箱熔断器、组件间连接和插头，修复开路点后使用电压表测试组串连续性并通过红外热像仪确认整体热斑消除，无需更换组件。

测试用例3 输出：
对这组图像进行分析，检测到4种缺陷：边缘积尘、草木遮挡、表面破损、内部缺陷。 需评估是否更换组件： 在可见光坐标(1399, 617)到(1598, 725)的范围检测到表面破损，需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。 在热成像坐标(557, 155)到(587, 165)的范围检测到内部缺陷，需评估电池片隐裂或断栅程度，若严重则需更换组件。 其他缺陷通常无需更换组件，只需修理： 对于草木遮挡，在可见光坐标(259, 345)到(289, 398)和可见光坐标(800, 900)到(850, 950)的范围，需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，无需更换组件。 对于边缘积尘，在可见光坐标(900, 1000)到(950, 1050)的范围，需要清除组件边缘积尘

### TypeOne_id7

In [ ]:
def TypeOne_id7(label):
    # 缺陷类型对应的影响描述
    impact_descriptions = {
        "组串短路": "可能导致部分或全部停发，可能造成发电系统局部或全部瘫痪。",
        "组串开路": "导致整串停发，可能造成发电系统局部或全部瘫痪。",
        "二极管故障": "热斑导致局部效率下降30%以上，可能造成发电系统局部瘫痪。",
        "内部缺陷": "效率下降10%-50%，可能造成发电系统局部瘫痪。",
        "组件异位": "角度偏移导致采光减少，效率下降10%-30%，通常不会导致系统瘫痪。",
        "表面破损": "透光率下降，效率下降10%-30%，通常不会导致系统瘫痪。",
        "草木遮挡": "遮挡区域发电归零并影响周边组件，效率下降10%-30%，通常不会导致系统瘫痪。",
        "固定设施遮挡": "局部小面积遮挡，效率下降小于10%，通常不会导致系统瘫痪。",
        "表面污迹": "透光率轻微下降，效率下降小于10%，通常不会导致系统瘫痪。",
        "边缘积尘": "边缘少量积尘，效率下降小于10%，通常不会导致系统瘫痪。"
    }

    faults = parse_label(label)

    # 检测到的缺陷类型
    fault_types = list(set(f["type"] for f in faults))
    if not fault_types:
        return "对这组图像进行分析，未检测到任何缺陷，发电系统正常运行，无瘫痪风险。"

    # 分组：严重影响（可能瘫痪）、中度影响、轻度影响
    severe_impact = ["组串短路", "组串开路", "二极管故障", "内部缺陷"]
    moderate_impact = ["组件异位", "表面破损", "草木遮挡"]
    low_impact = ["固定设施遮挡", "表面污迹", "边缘积尘"]

    severe_faults = [f for f in faults if f["type"] in severe_impact]
    moderate_faults = [f for f in faults if f["type"] in moderate_impact]
    low_faults = [f for f in faults if f["type"] in low_impact]

    # 生成回答
    answer_parts = []
    answer_parts.append(f"对这组图像进行分析，检测到{len(fault_types)}种缺陷：{'、'.join(fault_types)}。")

    # 处理严重影响缺陷（可能导致瘫痪）
    severe_desc = []
    for fault in severe_faults:
        fault_type = fault["type"]
        coords = fault["vis_coords"] or fault["ir_coords"] or "指定区域"
        coord_type = "可见光" if fault["vis_coords"] else "热成像"
        severe_desc.append(f"在{coord_type}{coords}的范围检测到{fault_type}，{impact_descriptions[fault_type]}")

    # 处理中度/轻度影响缺陷（不会瘫痪）
    moderate_low_desc = []
    moderate_low_types = list(set(f["type"] for f in moderate_faults + low_faults))
    for fault_type in moderate_low_types:
        coords_list = []
        for f in moderate_faults + low_faults:
            if f["type"] == fault_type:
                coord = f["vis_coords"] or f["ir_coords"] or "指定区域"
                coord_type = "可见光" if f["vis_coords"] else "热成像"
                coords_list.append(f"{coord_type}{coord}")
        if coords_list:
            coords_str = "和".join(coords_list)
            moderate_low_desc.append(f"对于{fault_type}，在{coords_str}的范围，{impact_descriptions[fault_type]}")

    # 综合描述
    if severe_desc:
        answer_parts.append("以下缺陷可能导致发电系统瘫痪：")
        answer_parts.extend(severe_desc)
        if moderate_low_desc:
            answer_parts.append("其他缺陷一般不会导致系统瘫痪，但会降低发电效率：")
            answer_parts.extend(moderate_low_desc)
    else:
        answer_parts.append("这种缺陷一般不会导致发电系统瘫痪，但会降低发电效率：")
        answer_parts.extend(moderate_low_desc if moderate_low_desc else ["未检测到影响发电的重大缺陷。"])

    return "".join(answer_parts)

print("测试用例1 输出：")
print(TypeOne_id7(label1))

print("\n测试用例2 输出：")
print(TypeOne_id7(label2))

print("\n测试用例3 输出：")
print(TypeOne_id7(label3))

print("测试用例4 输出：")
print(TypeOne_id7(label4))

测试用例1 输出：
对这组图像进行分析，检测到1种缺陷：表面破损。这种缺陷一般不会导致发电系统瘫痪，但会降低发电效率：对于表面破损，在可见光坐标(1399, 617)到(1598, 725)的范围，透光率下降，效率下降10%-30%，通常不会导致系统瘫痪。

测试用例2 输出：
对这组图像进行分析，检测到3种缺陷：草木遮挡、边缘积尘、组串开路。以下缺陷可能导致发电系统瘫痪：在热成像坐标(557, 155)到(587, 165)的范围检测到组串开路，导致整串停发，可能造成发电系统局部或全部瘫痪。其他缺陷一般不会导致系统瘫痪，但会降低发电效率：对于草木遮挡，在可见光坐标(259, 345)到(289, 398)的范围，遮挡区域发电归零并影响周边组件，效率下降10%-30%，通常不会导致系统瘫痪。对于边缘积尘，在可见光坐标(800, 900)到(850, 950)的范围，边缘少量积尘，效率下降小于10%，通常不会导致系统瘫痪。

测试用例3 输出：
对这组图像进行分析，检测到4种缺陷：边缘积尘、草木遮挡、表面破损、内部缺陷。以下缺陷可能导致发电系统瘫痪：在热成像坐标(557, 155)到(587, 165)的范围检测到内部缺陷，效率下降10%-50%，可能造成发电系统局部瘫痪。其他缺陷一般不会导致系统瘫痪，但会降低发电效率：对于边缘积尘，在可见光坐标(900, 1000)到(950, 1050)的范围，边缘少量积尘，效率下降小于10%，通常不会导致系统瘫痪。对于草木遮挡，在可见光坐标(259, 345)到(289, 398)和可见光坐标(800, 900)到(850, 950)的范围，遮挡区域发电归零并影响周边组件，效率下降10%-30%，通常不会导致系统瘫痪。对于表面破损，在可见光坐标(1399, 617)到(1598, 725)的范围，透光率下降，效率下降10%-30%，通常不会导致系统瘫痪。
测试用例4 输出：
对这组图像进行分析，未检测到任何缺陷，发电系统正常运行，无瘫痪风险。


### TypeOne_id12

In [ ]:
import re

def TypeOne_id12(label):
    # 固定科学说明部分，聚焦于遮挡但一般化以覆盖板子问题
    science_intro = (
        "这个问题需要通过红外图像的特征和可见光图像的对应迹象来判断。"
        "如果发生遮挡现象，红外图像上通常表现为区域光斑，这是因为遮挡会导致局部热分布不均，形成温度差异区域；"
        "在可见光图像中，则可以观察到草木或固定设施等外部物体迹象，从而确认是遮挡问题还是板子内部问题。"
    )

    # 缺陷特征描述字典
    feature_descriptions = {
        "草木遮挡": {"ir": "区域光斑", "vis": "草木", "category": "遮挡"},
        "固定设施遮挡": {"ir": "区域光斑或阴影", "vis": "固定设施或阴影", "category": "遮挡"},
        "表面污迹": {"ir": "区域光斑", "vis": "污渍", "category": "板子"},
        "表面破损": {"ir": "区域光斑", "vis": "破损", "category": "板子"},
        "内部缺陷": {"ir": "点状光斑", "vis": "无缺陷", "category": "板子"},
        "边缘积尘": {"ir": "区域光斑", "vis": "组件有边缘积尘", "category": "板子"},
        "组件异位": {"ir": "大面积区域光斑", "vis": "太阳光反射", "category": "板子"},
        "二极管故障": {"ir": "条状光斑", "vis": "无缺陷", "category": "板子"},
        "组串短路": {"ir": "光伏组件连续点状光斑", "vis": "无缺陷", "category": "板子"},
        "组串开路": {"ir": "光伏组件整体出现光斑", "vis": "无缺陷", "category": "板子"},
    }

    faults = parse_label(label)

    if not faults:
        analysis_part = "对这组图像进行分析，没有检测到缺陷特征，没有遮挡问题也没有板子问题。"
    else:
        analysis_part = "对这组图像进行分析，"
        descs = []
        for f in faults:
            fault_type = f["type"]
            if fault_type in feature_descriptions:
                ir_coords = f["ir_coords"]
                vis_coords = f["vis_coords"]
                ir_feature = feature_descriptions[fault_type]["ir"]
                vis_feature = feature_descriptions[fault_type]["vis"]
                category = feature_descriptions[fault_type]["category"]
                vis_desc = f"可见光图像{vis_coords}的范围识别到{vis_feature}" if vis_coords else f"可见光图像{vis_feature}"
                conclusion = "因此是遮挡问题而非板子问题" if category == "遮挡" else "因此是板子问题而非遮挡问题"
                desc = f"在红外图像{ir_coords}的区域，有{ir_feature}，而在{vis_desc}，可以辨别出是{fault_type}现象，{conclusion}"
                descs.append(desc)
        if descs:
            analysis_part += "；".join(descs) + "。"
        # else:
        #     analysis_part += "未检测到任何缺陷，因此无法明确辨别是遮挡问题还是板子问题。"

    answer = f"{science_intro} {analysis_part}"

    return answer

# 测试示例1：有遮挡
label1 = "总共有3个故障；故障1：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；故障2：内部缺陷，Bounding Box只在红外图像出现，坐标为(325, 282), (338, 291)；故障3：内部缺陷，Bounding Box只在红外图像出现，坐标为(532, 24), (621, 38)"
print("示例1输出：")
print(TypeOne_id12(label1))

# 测试示例2：有板子问题
label2 = "总共有2个故障；故障1：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)；故障2：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
print("\n示例2输出：")
print(TypeOne_id12(label2))

# 测试示例3：无相关缺陷或混合
label3 = "总共有1个故障；故障1：边缘积尘，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)"
print("\n示例3输出：")
print(TypeOne_id12(label3))

print("测试用例4 输出：")
print(TypeOne_id12(label4))

示例1输出：
这个问题需要通过红外图像的特征和可见光图像的对应迹象来判断。如果发生遮挡现象，红外图像上通常表现为区域光斑，这是因为遮挡会导致局部热分布不均，形成温度差异区域；在可见光图像中，则可以观察到草木或固定设施等外部物体迹象，从而确认是遮挡问题还是板子内部问题。 对这组图像进行分析，在红外图像坐标(73, 81)到(110, 108)的区域，有区域光斑，而在可见光图像坐标(259, 345)到(289, 398)的范围识别到草木，可以辨别出是草木遮挡现象，因此是遮挡问题而非板子问题；在红外图像坐标(325, 282)到(338, 291)的区域，有点状光斑，而在可见光图像无缺陷，可以辨别出是内部缺陷现象，因此是板子问题而非遮挡问题；在红外图像坐标(532, 24)到(621, 38)的区域，有点状光斑，而在可见光图像无缺陷，可以辨别出是内部缺陷现象，因此是板子问题而非遮挡问题。

示例2输出：
这个问题需要通过红外图像的特征和可见光图像的对应迹象来判断。如果发生遮挡现象，红外图像上通常表现为区域光斑，这是因为遮挡会导致局部热分布不均，形成温度差异区域；在可见光图像中，则可以观察到草木或固定设施等外部物体迹象，从而确认是遮挡问题还是板子内部问题。 对这组图像进行分析，在红外图像坐标(557, 155)到(587, 165)的区域，有点状光斑，而在可见光图像无缺陷，可以辨别出是内部缺陷现象，因此是板子问题而非遮挡问题；在红外图像坐标(365, 148)到(421, 176)的区域，有区域光斑，而在可见光图像坐标(1399, 617)到(1598, 725)的范围识别到破损，可以辨别出是表面破损现象，因此是板子问题而非遮挡问题。

示例3输出：
这个问题需要通过红外图像的特征和可见光图像的对应迹象来判断。如果发生遮挡现象，红外图像上通常表现为区域光斑，这是因为遮挡会导致局部热分布不均，形成温度差异区域；在可见光图像中，则可以观察到草木或固定设施等外部物体迹象，从而确认是遮挡问题还是板子内部问题。 对这组图像进行分析，在红外图像坐标(73, 81)到(110, 108)的区域，有区域光斑，而在可见光图像坐标(259, 345)到(289, 398)的范围识别到组件有边缘积尘，可以辨别出是边缘积尘现象，因此是板子问题而非遮挡问题。
测试用例4 输出：
这个问题需要通过红

### TypeOne_id13

In [ ]:
def TypeOne_id13(label):
    """
    这张照片中的热斑会造成上面影响?
    """
    # 固定科学说明部分
    science_intro = (
        "红外图像中的热斑是高温异常区域，通常由缺陷引起，不同缺陷会表现不同的热斑特征，可能导致发电效率下降、安全隐患或系统瘫痪等不同程度的影响。"
    )

    # 缺陷特征和影响描述字典
    descriptions = {
        "草木遮挡": {
            "ir_feature": "区域光斑",
            "vis_feature": "草木",
            "impact": "会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率"
        },
        "固定设施遮挡": {
            "ir_feature": "区域光斑或阴影",
            "vis_feature": "固定设施或阴影",
            "impact": "如果是局部小面积遮挡，效率下降小于10%，通常不会直接导致系统瘫痪"
        },
        "表面污迹": {
            "ir_feature": "区域光斑",
            "vis_feature": "污渍",
            "impact": "会引起透光率轻微下降，造成的发电效率下降一般小于10%，通常不会直接导致系统瘫痪"
        },
        "表面破损": {
            "ir_feature": "区域光斑",
            "vis_feature": "破损",
            "impact": "会导致透光率下降，致使10%-30%的发电效率下降，通常不会直接导致系统瘫痪"
        },
        "内部缺陷": {
            "ir_feature": "点状光斑",
            "vis_feature": "无明显特征",
            "impact": "影响较大，会导致发电效率下降10%-50%，可能造成发电系统局部瘫痪"
        },
        "边缘积尘": {
            "ir_feature": "区域光斑",
            "vis_feature": "边缘积尘",
            "impact": "边缘少量积尘，对整体透光影响较小，造成的发电效率下降一般小于10%，通常不会直接导致系统瘫痪"
        },
        "组件异位": {
            "ir_feature": "大面积区域光斑",
            "vis_feature": "太阳光反射",
            "impact": "角度偏移导致采光减少，效率下降10%-30%，通常不会导致系统瘫痪"
        },
        "二极管故障": {
            "ir_feature": "条状光斑",
            "vis_feature": "无明显特征",
            "impact": "热斑会导致局部效率下降30%以上，可能造成发电系统局部瘫痪"
        },
        "组串短路": {
            "ir_feature": "光伏组件连续点状光斑",
            "vis_feature": "无明显特征",
            "impact": "可能导致部分或全部停发，可能造成发电系统局部或全部瘫痪"
        },
        "组串开路": {
            "ir_feature": "光伏组件整体出现光斑",
            "vis_feature": "无明显特征",
            "impact": "导致整串停发，可能造成发电系统局部或全部瘫痪"
        },
    }

    # 解析label
    faults = parse_label(label)

    # 生成分析部分
    analysis_part = "对这组图像进行分析，"

    if not faults:
        analysis_part += "未检测到热斑相关特征，无明显影响。"
    else:
        descs = []
        for f in faults:
            fault_type = f["type"]
            if fault_type in descriptions:
                ir_coords = f["ir_coords"]
                vis_coords = f["vis_coords"]
                ir_feature = descriptions[fault_type]["ir_feature"]
                vis_feature = descriptions[fault_type]["vis_feature"]
                impact = descriptions[fault_type]["impact"]
                vis_desc = f"而在可见光图像对应区域有{vis_feature}" if vis_coords else f"而在可见光图像对应区域{vis_feature}"
                desc = f"在红外图像{ir_coords}的范围，存在{ir_feature}，{vis_desc}，对应{fault_type}的特征，{impact}"
                descs.append(desc)
        if descs:
            analysis_part += "；".join(descs) + "。"
        else:
            analysis_part += "未检测到热斑相关特征，无明显影响。"

    answer = f"{science_intro} {analysis_part}"

    return answer

# 测试用例1：严重影响（内部缺陷+组串开路）
label1 = (
    "总共有2个故障；故障1：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)；"
    "故障2：组串开路，Bounding Box只在红外图像出现，坐标为(532, 24), (621, 38)"
)
print("测试用例1 输出：")
print(TypeOne_id13(label1))

# 测试用例2：中度+轻度影响（表面破损+草木遮挡+边缘积尘）
label2 = (
    "总共有3个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)"
)
print("\n测试用例2 输出：")
print(TypeOne_id13(label2))

# 测试用例3：无热斑相关缺陷（边缘积尘）
label3 = (
    "总共有1个故障；故障1：边缘积尘，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)"
)
print("\n测试用例3 输出：")
print(TypeOne_id13(label3))

print("测试用例4 输出：")
print(TypeOne_id13(label4))

测试用例1 输出：
红外图像中的热斑是高温异常区域，通常由缺陷引起，不同缺陷会表现不同的热斑特征，可能导致发电效率下降、安全隐患或系统瘫痪等不同程度的影响。 对这组图像进行分析，在红外图像坐标(557, 155)到(587, 165)的范围，存在点状光斑，而在可见光图像对应区域无明显特征，对应内部缺陷的特征，影响较大，会导致发电效率下降10%-50%，可能造成发电系统局部瘫痪；在红外图像坐标(532, 24)到(621, 38)的范围，存在光伏组件整体出现光斑，而在可见光图像对应区域无明显特征，对应组串开路的特征，导致整串停发，可能造成发电系统局部或全部瘫痪。

测试用例2 输出：
红外图像中的热斑是高温异常区域，通常由缺陷引起，不同缺陷会表现不同的热斑特征，可能导致发电效率下降、安全隐患或系统瘫痪等不同程度的影响。 对这组图像进行分析，在红外图像坐标(365, 148)到(421, 176)的范围，存在区域光斑，而在可见光图像对应区域有破损，对应表面破损的特征，会导致透光率下降，致使10%-30%的发电效率下降，通常不会直接导致系统瘫痪；在红外图像坐标(73, 81)到(110, 108)的范围，存在区域光斑，而在可见光图像对应区域有草木，对应草木遮挡的特征，会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率；在红外图像坐标(300, 400)到(350, 450)的范围，存在区域光斑，而在可见光图像对应区域有边缘积尘，对应边缘积尘的特征，边缘少量积尘，对整体透光影响较小，造成的发电效率下降一般小于10%，通常不会直接导致系统瘫痪。

测试用例3 输出：
红外图像中的热斑是高温异常区域，通常由缺陷引起，不同缺陷会表现不同的热斑特征，可能导致发电效率下降、安全隐患或系统瘫痪等不同程度的影响。 对这组图像进行分析，在红外图像坐标(73, 81)到(110, 108)的范围，存在区域光斑，而在可见光图像对应区域有边缘积尘，对应边缘积尘的特征，边缘少量积尘，对整体透光影响较小，造成的发电效率下降一般小于10%，通常不会直接导致系统瘫痪。
测试用例4 输出：
红外图像中的热斑是高温异常区域，通常由缺陷引起，不同缺陷会表现不同的热斑特征，可能导致发电效率下降、安全隐患或系统瘫痪等不同程度的影响。 对这组图像进行

### TypeOne_id66

In [ ]:
def TypeOne_id66(label):
    """
    “从图上看，最要紧的问题是哪个呀？”
    """
    # 缺陷影响和优先级描述字典
    descriptions = {
        "组串短路": {
            "impact": "可能导致线路过热、设备烧毁甚至火灾，存在严重安全隐患",
            "priority": 1,
            "reason": "安全风险最高，可能引发火灾"
        },
        "组串开路": {
            "impact": "导致整串停止发电，显著降低系统输出",
            "priority": 2,
            "reason": "影响范围最大，需立即排查"
        },
        "二极管故障": {
            "impact": "引发热斑，导致局部效率下降30%以上，加速组件老化",
            "priority": 3,
            "reason": "热斑加速老化，需尽快处理"
        },
        "内部缺陷": {
            "impact": "效率下降10%-50%，可能导致局部瘫痪，故障持续恶化",
            "priority": 4,
            "reason": "隐性故障持续恶化，无法外部修复"
        },
        "组件异位": {
            "impact": "角度偏移减少采光，效率下降10%-30%，可能存在结构不稳定风险",
            "priority": 5,
            "reason": "结构不稳定需及时矫正"
        },
        "表面破损": {
            "impact": "透光率下降，效率下降10%-30%，可能因进水损坏内部电路",
            "priority": 6,
            "reason": "可能导致故障扩大，需尽快处理"
        },
        "草木遮挡": {
            "impact": "遮挡区域发电归零，效率下降10%-30%，影响随植物生长扩大",
            "priority": 7,
            "reason": "遮挡面积快速扩大，需定期清理"
        },
        "固定设施遮挡": {
            "impact": "局部小面积遮挡，效率下降小于10%，影响持续但固定",
            "priority": 8,
            "reason": "影响较小，可调整设施位置"
        },
        "表面污迹": {
            "impact": "透光率轻微下降，效率下降小于10%，无腐蚀性可按周期清理",
            "priority": 9,
            "reason": "影响轻微，可周期清理"
        },
        "边缘积尘": {
            "impact": "边缘少量积尘，效率下降小于10%，影响最小",
            "priority": 10,
            "reason": "影响最小，可常规清理"
        },
    }

    # 解析label
    faults = parse_label(label)

    # 生成回答
    if not faults:
        return "未检测到任何缺陷，无需优先处理任何问题。"

    # 检测到的缺陷类型
    fault_types = list(set(f["type"] for f in faults))
    answer_parts = [f"检测到{len(fault_types)}种缺陷：{'、'.join(fault_types)}。"]

    # 确定最要紧问题
    priority_faults = [(f["type"], descriptions.get(f["type"], {"priority": 99})["priority"], f["ir_coords"]) for f in faults]
    most_urgent = min(priority_faults, key=lambda x: x[1])
    urgent_type, urgent_priority, urgent_coords = most_urgent

    # 最要紧问题描述
    if urgent_type in descriptions:
        urgent_impact = descriptions[urgent_type]["impact"]
        urgent_reason = descriptions[urgent_type]["reason"]
        answer_parts.append(
            f"最要紧的问题是{urgent_type}，在红外图像{urgent_coords}的范围，"
            f"{urgent_impact}，因为{urgent_reason}。"
        )
    else:
        answer_parts.append("未检测到相关缺陷特征，无需优先处理任何问题。")
        return " ".join(answer_parts)

    # 其他缺陷描述，按优先级排序
    other_faults = [(f["type"], descriptions.get(f["type"], {"priority": 99})["priority"], f["ir_coords"])
                    for f in faults if f["type"] != urgent_type]
    if other_faults:
        answer_parts.append("其他缺陷的影响如下：")
        # 按优先级排序
        other_faults.sort(key=lambda x: x[1])
        prev_priority = None
        for fault_type, priority, coords in other_faults:
            if fault_type in descriptions:
                impact = descriptions[fault_type]["impact"]
                if prev_priority is None or priority <= 4:
                    desc = f"{fault_type}在红外图像{coords}的范围，{impact}。"
                elif priority <= 7:
                    desc = f"相较之下，{fault_type}在红外图像{coords}的范围，{impact}。"
                else:
                    desc = f"此外，{fault_type}在红外图像{coords}的范围，{impact}。"
                answer_parts.append(desc)
                prev_priority = priority

    return " ".join(answer_parts)

# 测试用例1：高优先级缺陷（组串短路+内部缺陷）
label1 = (
    "总共有2个故障；故障1：组串短路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)；"
    "故障2：内部缺陷，Bounding Box只在红外图像出现，坐标为(532, 24), (621, 38)"
)
print("测试用例1 输出：")
print(TypeOne_id66(label1))

# 测试用例2：中度+低优先级缺陷（表面破损+草木遮挡+边缘积尘）
label2 = (
    "总共有3个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)"
)
print("\n测试用例2 输出：")
print(TypeOne_id66(label2))

# 测试用例3：单一低优先级缺陷（边缘积尘）
label3 = (
    "总共有1个故障；故障1：边缘积尘，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)"
)
print("\n测试用例3 输出：")
print(TypeOne_id66(label3))

测试用例1 输出：
检测到2种缺陷：组串短路、内部缺陷。 最要紧的问题是组串短路，在红外图像坐标(557, 155)到(587, 165)的范围，可能导致线路过热、设备烧毁甚至火灾，存在严重安全隐患，因为安全风险最高，可能引发火灾。 其他缺陷的影响如下： 内部缺陷在红外图像坐标(532, 24)到(621, 38)的范围，效率下降10%-50%，可能导致局部瘫痪，故障持续恶化。

测试用例2 输出：
检测到3种缺陷：边缘积尘、草木遮挡、表面破损。 最要紧的问题是表面破损，在红外图像坐标(365, 148)到(421, 176)的范围，透光率下降，效率下降10%-30%，可能因进水损坏内部电路，因为可能导致故障扩大，需尽快处理。 其他缺陷的影响如下： 草木遮挡在红外图像坐标(73, 81)到(110, 108)的范围，遮挡区域发电归零，效率下降10%-30%，影响随植物生长扩大。 此外，边缘积尘在红外图像坐标(300, 400)到(350, 450)的范围，边缘少量积尘，效率下降小于10%，影响最小。

测试用例3 输出：
检测到1种缺陷：边缘积尘。 最要紧的问题是边缘积尘，在红外图像坐标(73, 81)到(110, 108)的范围，边缘少量积尘，效率下降小于10%，影响最小，因为影响最小，可常规清理。


### TypeOne_id14——**修复这块光伏板的热斑问题需要带什么工具？**

In [ ]:
def TypeOne_id14(label):
    """
    “修复这块光伏板的热斑问题需要带什么工具？”
    """
    # 缺陷优先级和工具提取
    defect_info = {
        "组串短路": {
            "tools": ["汇流箱熔断器", "绝缘电阻测试仪", "红外热像仪"],
            "strategy": "需要先使用汇流箱熔断器断开短路组串回路，排查并修复短路点，然后用绝缘电阻测试仪验证无短路，最后用红外热像仪确认连续点状热斑消失，一般无需更换组件",
            "priority": 1
        },
        "组串开路": {
            "tools": ["电压表", "红外热像仪"],
            "strategy": "需要先检查汇流箱熔断器、组件间连接和插头，修复开路点后用电压表测试组串连续性，最后用红外热像仪确认整体热斑消除，一般无需更换组件",
            "priority": 2
        },
        "二极管故障": {
            "tools": ["多用表", "红外热像仪", "备用二极管"],
            "strategy": "需要先检查接线盒，用多用表测试二极管导通，若可更换则使用备用二极管修复，之后用红外热像仪确认条状热斑消除；若不可更换，则需考虑更换光伏组件",
            "priority": 3
        },
        "内部缺陷": {
            "tools": ["EL电致发光测试仪", "红外热像仪", "I-V曲线测试仪"],
            "strategy": "需要先用EL电致发光测试仪、红外热像仪或I-V曲线测试仪确认缺陷范围和功率衰减，若严重则需考虑更换光伏组件",
            "priority": 4
        },
        "组件异位": {
            "tools": ["螺丝刀", "红外热像仪", "I-V曲线测试仪"],
            "strategy": "先用螺丝刀重新固定或对齐异位组件，确保支架稳固，然后用红外热像仪检查大面积热斑消失，最后用I-V曲线测试仪验证电压电流平衡，一般无需更换组件",
            "priority": 5
        },
        "表面破损": {
            "tools": ["专用修补胶", "密封剂", "红外热像仪", "电绝缘测试仪"],
            "strategy": "需要先评估破损程度，若轻微划痕则用专用修补胶或密封剂修复玻璃或背板，之后用红外热像仪和电绝缘测试仪检查无热斑或漏电风险；若严重则需考虑更换光伏组件",
            "priority": 6
        },
        "草木遮挡": {
            "tools": ["镰刀", "剪刀", "I-V曲线测试仪", "红外热像仪"],
            "strategy": "需要先用镰刀或剪刀清除杂草和树枝，然后用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，一般无需更换组件",
            "priority": 7
        },
        "固定设施遮挡": {
            "tools": ["螺丝刀", "红外热像仪", "I-V曲线测试仪"],
            "strategy": "需要先用螺丝刀移除或调整固定设施如支架或围栏，若无法移除则重新布局组件，然后用红外热像仪验证阴影消除，最后用I-V曲线测试仪确认发电效率恢复，一般无需更换组件",
            "priority": 8
        },
        "表面污迹": {
            "tools": ["水", "中性清洁剂", "红外热像仪", "I-V曲线测试仪"],
            "strategy": "需要先用水或中性清洁剂去除污渍，然后用红外热像仪复测组件，若热斑仍存在则用I-V曲线测试仪检测功率，若功率低于期望值则需考虑更换组件",
            "priority": 9
        },
        "边缘积尘": {
            "tools": ["清洁工具", "红外热像仪"],
            "strategy": "需要先用清洁工具清除边缘积尘，然后用红外热像仪复测组件，一般无需更换组件",
            "priority": 10
        },
    }

    # 解析label
    faults = parse_label(label)
    print(faults)

    # 生成回答
    if not faults:
        return "对这组图像进行分析，未检测到任何热斑相关缺陷，无需携带工具进行修复。"

    # 检测到的缺陷类型
    fault_types = list(set(f["type"] for f in faults))
    answer_parts = [f"对这组图像进行分析，共检测到{len(fault_types)}种缺陷：{'、'.join(fault_types)}。修复热斑问题需携带以下工具："]

    # 按优先级排序故障
    priority_faults = [(f["type"], defect_info.get(f["type"], {"priority": 99})["priority"], f["ir_coords"])
                      for f in faults]
    priority_faults.sort(key=lambda x: x[1])

    # 描述每个缺陷的工具需求
    tool_descs = []
    all_tools = set()
    for fault_type, priority, ir_coords in priority_faults:
        if fault_type in defect_info:
            tools = defect_info[fault_type]["tools"]
            strategy = defect_info[fault_type]["strategy"]
            all_tools.update(tools)
            if priority <= 4:  # 高优先级缺陷，详细描述
                desc = f"对于出现在红外图像{ir_coords}范围的{fault_type}，{strategy}，需携带{'、'.join(tools)}。"
            else:  # 中低优先级，简述
                desc = f"对于出现在红外图像{ir_coords}范围的{fault_type}，{strategy}，需携带{'、'.join(tools)}。"
            tool_descs.append(desc)

    # 综合工具列表
    answer_parts.extend(tool_descs)
    answer_parts.append(f"总结所需工具有{'、'.join(sorted(all_tools))}。")


    return "".join(answer_parts)

# 测试用例1：高优先级缺陷（组串短路+内部缺陷）
label1 = (
    "总共有2个故障；故障1：组串短路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)；"
    "故障2：内部缺陷，Bounding Box只在红外图像出现，坐标为(532, 24), (621, 38)"
)
print("测试用例1 输出：")
print(TypeOne_id14(label1))

# 测试用例2：中度+低优先级缺陷（表面破损+草木遮挡+边缘积尘）
label2 = (
    "总共有3个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)"
)
print("\n测试用例2 输出：")
print(TypeOne_id14(label2))

# 测试用例3：单一低优先级缺陷（边缘积尘）
label3 = (
    "总共有1个故障；故障1：边缘积尘，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)"
)
print("\n测试用例3 输出：")
print(TypeOne_id14(label3))

测试用例1 输出：
[{'type': '组串短路', 'ir_coords': '坐标(557, 155)到(587, 165)', 'vis_coords': None}, {'type': '内部缺陷', 'ir_coords': '坐标(532, 24)到(621, 38)', 'vis_coords': None}]
对这组图像进行分析，共检测到2种缺陷：组串短路、内部缺陷。修复热斑问题需携带以下工具：对于出现在红外图像坐标(557, 155)到(587, 165)范围的组串短路，需要先使用汇流箱熔断器断开短路组串回路，排查并修复短路点，然后用绝缘电阻测试仪验证无短路，最后用红外热像仪确认连续点状热斑消失，一般无需更换组件，需携带汇流箱熔断器、绝缘电阻测试仪、红外热像仪。对于出现在红外图像坐标(532, 24)到(621, 38)范围的内部缺陷，需要先用EL电致发光测试仪、红外热像仪或I-V曲线测试仪确认缺陷范围和功率衰减，若严重则需考虑更换光伏组件，需携带EL电致发光测试仪、红外热像仪、I-V曲线测试仪。总结所需工具有EL电致发光测试仪、I-V曲线测试仪、汇流箱熔断器、红外热像仪、绝缘电阻测试仪。

测试用例2 输出：
[{'type': '表面破损', 'ir_coords': '坐标(365, 148)到(421, 176)', 'vis_coords': '坐标(1399, 617)到(1598, 725)'}, {'type': '草木遮挡', 'ir_coords': '坐标(73, 81)到(110, 108)', 'vis_coords': '坐标(259, 345)到(289, 398)'}, {'type': '边缘积尘', 'ir_coords': '坐标(300, 400)到(350, 450)', 'vis_coords': '坐标(900, 1000)到(950, 1050)'}]
对这组图像进行分析，共检测到3种缺陷：边缘积尘、草木遮挡、表面破损。修复热斑问题需携带以下工具：对于出现在红外图像坐标(365, 148)到(421, 176)范围的表面破损，需要先评估破损程度，若轻微划痕则用专用修补胶或密封剂修复玻璃或背板，之后用红外热像仪和电绝缘测试仪检查无热斑或漏电风险；若严重则需考虑更换光伏组件，需携带专用修补胶、密封剂、红

### TypeOne_id14--**哪些问题是需要最优先处理的？**

In [ ]:
def TypeOne_id22(label):

    # 缺陷特征、缺陷影响、处理优先级、处理原因、维修策略描述
    descriptions = {
        "组串短路": {
            "ir_feature": "连续点状热斑",
            "vis_feature": "无明显特征",
            "impact": "会导致光伏面板部分或全部停发，因线路过热形成严重热斑，存在设备烧毁或火灾的潜在风险",
            "peocess priority": 1,
            "reason": "安全风险最高，需要立即处理避免火灾发生",
            "repair_strategy": "需要通过断开汇流箱熔断器断开短路组串回路，排查并修复短路故障点，然后使用绝缘电阻测试仪验证无短路并通过红外热像仪确认连续点状热斑消失，通常无需更换组件。",
        },
        "组串开路": {
            "ir_feature": "整体热斑",
            "vis_feature": "无明显特征",
            "impact": "会直接导致整个组串停止发电，形成大范围热斑，显著降低系统发电效率",
            "peocess priority": 2,
            "reason": "影响范围最大，需立即排查",
            "repair_strategy": "需要检查汇流箱熔断器、组件间连接和插头，修复开路点后使用电压表测试组串连续性并通过红外热像仪确认整体热斑消除，通常无需更换组件。"
        },
        "二极管故障": {
            "ir_feature": "条状热斑",
            "vis_feature": "无明显特征",
            "impact": "因热斑效应导致局部发电效率下降30%以上，存在加速组件老化的潜在风险",
            "peocess priority": 3,
            "reason": "可能加速老化，需尽快处理",
            "repair_strategy": "需要检查接线盒，若二极管可单独更换则进行更换，然后使用多用表测试二极管导通并通过红外热像仪确认条状热斑消除；若二极管不可单独更换或更换成本过高，则需更换组件。",
        },
        "内部缺陷": {
            "ir_feature": "点状热斑",
            "vis_feature": "无明显特征",
            "impact": "属于隐性故障，会降低发电效率下降10%-50%，而且会持续恶化，存在局部瘫痪的潜在风险",
            "peocess priority": 4,
            "reason": "故障持续恶化，无法外部修复",
            "repair_strategy": "需要使用EL电致发光测试、红外热像仪或I-V曲线分析确认缺陷范围和功率衰减程度，若为电池片隐裂、断栅、焊带脱落等严重问题，则需更换组件。",
        },
        "组件异位": {
            "ir_feature": "大面积区域热斑",
            "vis_feature": "太阳光反射",
            "impact": "因角度偏移导致采光受到影响，可能有10%-30%的发电效率下降，存在结构不稳定的潜在风险",
            "peocess priority": 5,
            "reason": "结构不稳定，有掉落风险，需及时矫正",
            "repair_strategy": "需要重新固定或对齐异位组件，确保安装支架稳固，然后使用红外热像仪检查大面积热斑消失并测试组串电压电流平衡，通常无需更换组件。",
        },
        "表面破损": {
            "ir_feature": "区域热斑",
            "vis_feature": "破损现象",
            "impact": "因破损导致透光率下降，可能导致发电效率下降10%-30%，存在进水损坏电路的潜在风险",
            "peocess priority": 6,
            "reason": "玻璃破碎后会进水、进尘，损坏内部电路，需尽快处理以防止故障扩大。",
            "repair_strategy": "需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。",
        },
        "草木遮挡": {
            "ir_feature": "区域热斑",
            "vis_feature": "草木",
            "impact": "会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率",
            "peocess priority": 7,
            "reason": "遮挡面积会随植物生长扩大，快速降低局部发电效率，需定期清理。",
            "repair_strategy": "需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，通常无需更换组件。",
        },
        "固定设施遮挡": {
            "ir_feature": "区域热斑或阴影",
            "vis_feature": "固定设施或阴影",
            "impact": "因固定设施导致光伏板局部面积遮挡，通常引起的发电效率下降小于10%",
            "peocess priority": 8,
            "reason": "遮挡位置固定但持续影响，潜在风险较低，需通过调整设施角度、位置来解决",
            "repair_strategy": "需要评估并移除或调整固定设施如支架或围栏，若无法移除则重新布局组件位置，然后使用红外热像仪验证阴影消除并进行I-V曲线测试确认发电效率恢复，通常无需更换组件。",
        },
        "表面污迹": {
            "ir_feature": "区域热斑",
            "vis_feature": "污渍",
            "impact": "会轻微影响光伏板透光率，通常引起的发电效率下降小于10%",
            "peocess priority": 9,
            "reason": "潜在风险低，若污迹干燥且无腐蚀性，可按周期清理",
            "repair_strategy": "需要使用水或中性清洁剂去除组件表面污渍，然后在辐照度较好的环境下，用红外设备复测组件，如热斑依然存在，使用I-V曲线测试仪检测组件功率，若功率低于期望值，则需更换组件。",
        },
        "边缘积尘": {
            "ir_feature": "区域热斑",
            "vis_feature": "边缘积尘",
            "impact": "因积尘量少，污染程度较低，通常引起的发电效率下降小于10%",
            "peocess priority": 10,
            "reason": "影响相对较小，可在常规清洁时顺带处理",
            "repair_strategy": "需要清除组件边缘积尘，然后通过红外设备复测组件，通常无需更换组件。如现象普遍，应对全站组件边缘进行重点清洗。"
        },
    }

    # 解析label
    faults = parse_label(label)

    # 生成回答
    if not faults:
        return "未检测到任何异常情况，无需优先处理任何问题。"

    # 检测到的缺陷类型
    fault_types = list(set(f["type"] for f in faults))
    answer_parts = [f"对这组图像进行分析，共检测到{len(fault_types)}种异常情况：{'、'.join(fault_types)}。"]

    # 按优先级排序故障
    priority_faults = [(f["type"], descriptions.get(f["type"], {"peocess priority": 99})["peocess priority"],
                       f["ir_coords"], f["vis_coords"]) for f in faults]
    priority_faults.sort(key=lambda x: x[1])

    # 最优先问题
    most_urgent = priority_faults[0]
    urgent_type, urgent_priority, urgent_ir_coords, urgent_vis_coords = most_urgent
    if urgent_type in descriptions:
        ir_feature = descriptions[urgent_type]["ir_feature"]
        vis_feature = descriptions[urgent_type]["vis_feature"]
        impact = descriptions[urgent_type]["impact"]
        reason = descriptions[urgent_type]["reason"]
        vis_desc = f"可见光图像{urgent_vis_coords}呈现{vis_feature}" if urgent_vis_coords else f"可见光图像{vis_feature}"
        answer_parts.append(
            f"其中，最需要优先处理的是{urgent_type}，在红外图像{urgent_ir_coords}的范围形成{ir_feature}，"
            f"而在{vis_desc}，{impact}。这需要优先处理，因为{reason}。"
        )

    # 其他缺陷描述
    other_faults = priority_faults[1:]
    if other_faults:
        answer_parts.append("其他异常情况的影响如下：")
        for fault_type, priority, ir_coords, vis_coords in other_faults:
            if fault_type in descriptions:
                ir_feature = descriptions[fault_type]["ir_feature"]
                vis_feature = descriptions[fault_type]["vis_feature"]
                impact = descriptions[fault_type]["impact"]
                vis_desc = f"可见光图像{vis_coords}呈现{vis_feature}" if vis_coords else f"可见光图像{vis_feature}"
                if priority <= 7:
                    desc = f"相较之下，{fault_type}在红外图像{ir_coords}的范围形成{ir_feature}，而在{vis_desc}，{impact}。"
                else:
                    desc = f"此外，{fault_type}在红外图像{ir_coords}的范围形成{ir_feature}，而在{vis_desc}，{impact}。"
                answer_parts.append(desc)

    return " ".join(answer_parts)

# 测试用例1：高优先级缺陷（组串短路+内部缺陷）
label1 = (
    "总共有2个故障；故障1：组串短路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)；"
    "故障2：内部缺陷，Bounding Box只在红外图像出现，坐标为(532, 24), (621, 38)"
)
print("测试用例1 输出：")
print(TypeOne_id22(label1))

# 测试用例2：中度+低优先级缺陷（表面破损+草木遮挡+边缘积尘）
label2 = (
    "总共有3个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)"
)
print("\n测试用例2 输出：")
print(TypeOne_id22(label2))

# 测试用例3：单一低优先级缺陷（边缘积尘）
label3 = (
    "总共有1个故障；故障1：边缘积尘，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)"
)
print("\n测试用例3 输出：")
print(TypeOne_id22(label3))

测试用例1 输出：
对这组图像进行分析，共检测到2种异常情况：组串短路、内部缺陷。 其中，最需要优先处理的是组串短路，在红外图像坐标(557, 155)到(587, 165)的范围形成连续点状热斑，而在可见光图像无明显特征，会导致光伏面板部分或全部停发，因线路过热形成严重热斑，存在设备烧毁或火灾的潜在风险。这需要优先处理，因为安全风险最高，需要立即处理避免火灾发生。 其他异常情况的影响如下： 相较之下，内部缺陷在红外图像坐标(532, 24)到(621, 38)的范围形成点状热斑，而在可见光图像无明显特征，属于隐性故障，会降低发电效率下降10%-50%，而且会持续恶化，存在局部瘫痪的潜在风险。

测试用例2 输出：
对这组图像进行分析，共检测到3种异常情况：边缘积尘、草木遮挡、表面破损。 其中，最需要优先处理的是表面破损，在红外图像坐标(365, 148)到(421, 176)的范围形成区域热斑，而在可见光图像坐标(1399, 617)到(1598, 725)呈现破损现象，因破损导致透光率下降，可能导致发电效率下降10%-30%，存在进水损坏电路的潜在风险。这需要优先处理，因为玻璃破碎后会进水、进尘，损坏内部电路，需尽快处理以防止故障扩大。。 其他异常情况的影响如下： 相较之下，草木遮挡在红外图像坐标(73, 81)到(110, 108)的范围形成区域热斑，而在可见光图像坐标(259, 345)到(289, 398)呈现草木，会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率。 此外，边缘积尘在红外图像坐标(300, 400)到(350, 450)的范围形成区域热斑，而在可见光图像坐标(900, 1000)到(950, 1050)呈现边缘积尘，因积尘量少，污染程度较低，通常引起的发电效率下降小于10%。

测试用例3 输出：
对这组图像进行分析，共检测到1种异常情况：边缘积尘。 其中，最需要优先处理的是边缘积尘，在红外图像坐标(73, 81)到(110, 108)的范围形成区域热斑，而在可见光图像坐标(259, 345)到(289, 398)呈现边缘积尘，因积尘量少，污染程度较低，通常引起的发电效率下降小于10%。这需要优先处理，因为影响相对较小，可在常规清洁时顺带处理。


### TypeOne_id22
**根据这些照片显示的异常情况（热斑温差、遮挡面积、污染程度、潜在风险），哪些问题是需要最优先处理的？**

In [ ]:
def TypeOne_id22(label):
    if label == "没有缺陷":
        return "结合这组图像进行分析，没有缺陷。在这个热斑位置的可见光照片上，没有看到明显的异常，如裂纹、破片、污渍、遮挡物或阴影。"

    faults = parse_label(label)
    total_faults = len(faults)

    if total_faults == 0:
        return "结合这组图像进行分析，没有缺陷。在这个热斑位置的可见光照片上，没有看到明显的异常，如裂纹、破片、污渍、遮挡物或阴影。"

    answer_parts = [f"结合这组图像进行分析，共有{total_faults}处缺陷。其中，"]

    fault_types = list(set(f["type"] for f in faults))

    for fault_type in fault_types:
        instances = [f for f in faults if f['type'] == fault_type]
        ir_feature = descriptions.get(fault_type, {}).get('ir_feature', '未知热斑特征')
        vis_feature = descriptions.get(fault_type, {}).get('vis_feature', '未知异常')

        # 根据缺陷类型设置可见光描述
        if fault_type == "表面破损":
            vis_desc = "裂纹或破片等破损现象"
        elif fault_type == "草木遮挡":
            vis_desc = "树叶或草木等遮挡物"
        elif fault_type == "边缘积尘":
            vis_desc = "灰尘堆积"
        elif fault_type == "表面污迹":
            vis_desc = "污渍如鸟粪或泥土"
        elif fault_type == "固定设施遮挡":
            vis_desc = "固定设施或阴影"
        elif fault_type == "组件异位":
            vis_desc = "明显的太阳光反射"
        else:
            vis_desc = vis_feature

        ir_coords_list = [inst['ir_coords'] for inst in instances]
        vis_coords_list = [inst['vis_coords'] if inst['vis_coords'] else None for inst in instances]

        has_vis = instances[0]['vis_coords'] is not None

        if not has_vis:
            vis_desc = vis_feature  # 通常为"无明显特征"

        if len(ir_coords_list) > 1:
            ir_str = "、".join(ir_coords_list[:-1]) + "和" + ir_coords_list[-1]
            if has_vis:
                vis_str = "、".join(vis_coords_list[:-1]) + "和" + vis_coords_list[-1]
                answer_parts.append(f"多处{fault_type}缺陷，体现在热成像上{ir_str}的范围，呈现出{ir_feature}特征，在可见光对应{vis_str}的区域，能看到{vis_desc}。")
            else:
                answer_parts.append(f"多处{fault_type}缺陷，体现在热成像上{ir_str}的范围，呈现出{ir_feature}特征，在可见光区域无明显特征。")
        else:
            ir_str = ir_coords_list[0]
            if has_vis:
                vis_str = vis_coords_list[0]
                answer_parts.append(f"{fault_type}缺陷，体现在热成像上{ir_str}的范围，呈现出{ir_feature}特征，在可见光对应{vis_str}的区域，能看到{vis_desc}。")
            else:
                answer_parts.append(f"{fault_type}缺陷，体现在热成像上{ir_str}的范围，呈现出{ir_feature}特征，在可见光上对应{vis_str}的区域。")

    return "".join(answer_parts)

print("测试用例1 输出：")
print(TypeOne_id22(label1))

print("\n测试用例2 输出：")
print(TypeOne_id22(label2))

print("\n测试用例3 输出：")
print(TypeOne_id22(label3))

print("测试用例4 输出：")
print(TypeOne_id22(label4))

测试用例1 输出：
结合这组图像进行分析，共有1处缺陷。其中，表面破损缺陷，体现在热成像上坐标(365, 148)到(421, 176)的范围，呈现出区域热斑特征，在可见光对应坐标(1399, 617)到(1598, 725)的区域，能看到裂纹或破片等破损现象。

测试用例2 输出：
结合这组图像进行分析，共有3处缺陷。其中，草木遮挡缺陷，体现在热成像上坐标(73, 81)到(110, 108)的范围，呈现出区域热斑特征，在可见光对应坐标(259, 345)到(289, 398)的区域，能看到树叶或草木等遮挡物。边缘积尘缺陷，体现在热成像上坐标(200, 300)到(250, 350)的范围，呈现出区域热斑特征，在可见光对应坐标(800, 900)到(850, 950)的区域，能看到灰尘堆积。组串开路缺陷，体现在热成像上坐标(557, 155)到(587, 165)的范围，呈现出整体热斑特征，在可见光上对应坐标(800, 900)到(850, 950)的区域。

测试用例3 输出：
结合这组图像进行分析，共有5处缺陷。其中，多处草木遮挡缺陷，体现在热成像上坐标(73, 81)到(110, 108)和坐标(200, 300)到(250, 350)的范围，呈现出区域热斑特征，在可见光对应坐标(259, 345)到(289, 398)和坐标(800, 900)到(850, 950)的区域，能看到树叶或草木等遮挡物。边缘积尘缺陷，体现在热成像上坐标(300, 400)到(350, 450)的范围，呈现出区域热斑特征，在可见光对应坐标(900, 1000)到(950, 1050)的区域，能看到灰尘堆积。内部缺陷缺陷，体现在热成像上坐标(557, 155)到(587, 165)的范围，呈现出点状热斑特征，在可见光上对应坐标(900, 1000)到(950, 1050)的区域。表面破损缺陷，体现在热成像上坐标(365, 148)到(421, 176)的范围，呈现出区域热斑特征，在可见光对应坐标(1399, 617)到(1598, 725)的区域，能看到裂纹或破片等破损现象。
测试用例4 输出：
结合这组图像进行分析，没有缺陷。在这个热斑位置的可见光照片上，没有看到明显的异常，如裂纹、破片、污渍、遮挡物或阴影。


### TypeOne_id26
**红外图像中某区域显示高温异常，具体可能是什么故障？（区分热斑、接线故障或灰尘遮挡）**

In [ ]:
def TypeOne_id26(label):
    if label == "没有缺陷":
        return "结合这组图像进行分析，在红外图像中未显示高温异常，没有发现明显缺陷。"

    faults = parse_label(label)
    total_faults = len(faults)
    if total_faults == 0:
        return "结合这组图像进行分析，在红外图像中未显示高温异常，没有发现明显缺陷。"

    answer_parts = [f"结合这组图像进行分析，共有{total_faults}处缺陷，可能导致红外图像中对应区域显示高温异常。"]

    # 原映射（保留以兜底）
    categories = {
        "热斑类故障": ["内部缺陷", "二极管故障"],
        "接线故障": ["组串短路", "组串开路"],
        "灰尘或遮挡类故障": ["草木遮挡", "固定设施遮挡", "表面污迹", "边缘积尘"]
    }
    # 简洁类别名（用于成文落款）
    concise_cat = {
        "草木遮挡": "遮挡类故障",
        "固定设施遮挡": "遮挡类故障",
        "表面污迹": "灰尘类故障",
        "边缘积尘": "灰尘类故障",
        "表面破损": "表面损坏类故障",
        "内部缺陷": "热斑类故障",
        "二极管故障": "热斑类或接线类故障",
        "组串短路": "接线类故障",
        "组串开路": "接线类故障",
        "组件异位": "结构异位类故障",
    }

    fault_types = list(set(f["type"] for f in faults))
    num_word = lambda n: "1处" if n == 1 else ("2处" if n == 2 else f"{n}处")

    for fault_type in fault_types:
        if fault_type == "二极管故障":
          ir_str = "和".join(ir_coords_list) if len(ir_coords_list) > 1 else ir_coords_list[0]
          answer_parts.append(
              f"{num_word(len(instances))}二极管故障出现在热成像上{ir_str}的范围，"
              f"呈现出条状热斑。该类故障可能由两类原因引起："
              f"一是外部线路存在虚接、过载等问题，导致二极管过流烧毁；"
              f"二是内置二极管自身老化或制造缺陷所致。"
              f"前者属于接线类故障，后者属于热斑类故障。"
          )
          continue
        instances = [f for f in faults if f['type'] == fault_type]
        ir_feature = descriptions.get(fault_type, {}).get('ir_feature', '未知热斑特征')
        cause_raw = descriptions.get(fault_type, {}).get('cause', '')

        cause_core = cause_raw.lstrip("其").lstrip("由于").strip("，。 　")
        cause_phrase = f"其是由于{cause_core}" if cause_core else ""

        fallback_cat = "其他故障"
        for cat, types in categories.items():
            if fault_type in types:
                fallback_cat = cat
                break
        cat_text = concise_cat.get(fault_type, fallback_cat)

        ir_coords_list = [inst['ir_coords'].replace('坐标', '') for inst in instances]
        ir_str = "和".join(ir_coords_list) if len(ir_coords_list) > 1 else ir_coords_list[0]

        # 成文模板：数量 + 名称 + 出现位置 + 形态 + 原因 + 类别
        answer_parts.append(
            f"{num_word(len(instances))}{fault_type}出现在热成像上{ir_str}的范围，呈现出{ir_feature}，{cause_phrase}，属于{cat_text}。"
            .replace("，，", "，").replace("。，", "。")
        )

    return "".join(answer_parts)


# 测试用例1
label1 = (
    "总共有1个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
)
print("测试用例1:", TypeOne_id22(label1))

# 测试用例2
label2 = (
    "总共有3个故障；故障1：二极管故障，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障2：边缘积尘，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障3：组串开路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("测试用例2:", TypeOne_id22(label2))

# 测试用例3
label3 = (
    "总共有5个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：草木遮挡，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障4：草木遮挡，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)；"
    "故障5：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("测试用例3:", TypeOne_id22(label3))

# 测试用例4
label4 = (
    "没有缺陷"
)
print("测试用例4:", TypeOne_id22(label4))

测试用例1: 结合这组图像进行分析，共有1处缺陷。其中，表面破损缺陷，体现在热成像上坐标(365, 148)到(421, 176)的范围，呈现出区域热斑特征，在可见光对应坐标(1399, 617)到(1598, 725)的区域，能看到裂纹或破片等破损现象。
测试用例2: 结合这组图像进行分析，共有3处缺陷。其中，边缘积尘缺陷，体现在热成像上坐标(200, 300)到(250, 350)的范围，呈现出区域热斑特征，在可见光对应坐标(800, 900)到(850, 950)的区域，能看到灰尘堆积。二极管故障缺陷，体现在热成像上坐标(73, 81)到(110, 108)的范围，呈现出条状热斑特征，在可见光对应坐标(259, 345)到(289, 398)的区域，能看到无明显特征。组串开路缺陷，体现在热成像上坐标(557, 155)到(587, 165)的范围，呈现出整体热斑特征，在可见光区域无明显特征。
测试用例3: 结合这组图像进行分析，共有5处缺陷。其中，多处草木遮挡缺陷，体现在热成像上坐标(73, 81)到(110, 108)、坐标(200, 300)到(250, 350)和坐标(300, 400)到(350, 450)的范围，呈现出区域热斑特征，在可见光对应坐标(259, 345)到(289, 398)、坐标(800, 900)到(850, 950)和坐标(900, 1000)到(950, 1050)的区域，能看到树叶或草木等遮挡物。表面破损缺陷，体现在热成像上坐标(365, 148)到(421, 176)的范围，呈现出区域热斑特征，在可见光对应坐标(1399, 617)到(1598, 725)的区域，能看到裂纹或破片等破损现象。内部缺陷缺陷，体现在热成像上坐标(557, 155)到(587, 165)的范围，呈现出点状热斑特征，在可见光区域无明显特征。
测试用例4: 结合这组图像进行分析，没有缺陷。在这个热斑位置的可见光照片上，没有看到明显的异常，如裂纹、破片、污渍、遮挡物或阴影。


### TypeOne_id47
**红外照片中显示的异常热点区域具体位置是哪里？**

In [ ]:
# 红外照片中显示的异常热点区域具体位置是哪里？
def TypeOne_id47(label):
    f = [x for x in parse_label(label) if x.get('ir_coords')]
    if not f: return "对这组图像进行分析，未检测到红外照片中的异常热点区域，该光伏板没有明显缺陷"
    g, n = {}, len(f)
    for x in f: g.setdefault(x['type'], []).append(x['ir_coords'])
    num = lambda k: "一处" if k==1 else ("两处" if k==2 else f"{k}处")
    join = lambda L: L[0] if len(L)==1 else ("和".join(L) if len(L)==2 else "、".join(L[:-1])+"和"+L[-1])
    if n==1:
        t, L = next(iter(g.items()))
        return f"对这组图像进行分析，共检测到1处异常情况，为{t}，表现为{descriptions.get(t,{}).get('ir_feature','未知特征')}，位于红外图像的{L[0]}的范围。这些热点区域表明光伏组件存在潜在问题，需要进一步检查。"
    s=[f"有{num(len(L))}{t}，红外图像上表现为{descriptions.get(t,{}).get('ir_feature','未知特征')}，位于坐标{join(L)}的范围" for t,L in g.items()]
    return f"对这组图像进行分析，共检测到{n}处异常情况：" + "；".join(s) + "；这些热点区域表明光伏组件存在潜在问题，需要进一步检查。"

print("测试用例1 输出：")
print(TypeOne_id47(label1))
print("\n测试用例2 输出：")
print(TypeOne_id47(label2))
print("\n测试用例3 输出：")
print(TypeOne_id47(label3))
print("测试用例4 输出：")
print(TypeOne_id47(label4))

SyntaxError: '(' was never closed (ipython-input-2296909238.py, line 18)

### TypeOne_id59
**这个有问题的地方影响发电不？**

In [ ]:
# 这个有问题的地方影响发电不？
def generate_answer_impact_summary(label):
    f = parse_label(label)
    if not f: return "对这组图像进行分析，未检测到异常情况。结论：不影响发电。"
    # 聚合各缺陷的数量与严重度
    s = {}
    for x in f:
        t = x['type']; d = descriptions.get(t, {})
        s.setdefault(t, {"cnt":0, "p": d.get("peocess priority", 999), "impact": d.get("impact","可能影响发电效率"), "reason": d.get("reason","")})
        s[t]["cnt"] += 1
    types = sorted(s.items(), key=lambda kv: kv[1]["p"])
    T, N = len(types), len(f)
    top_name, top = types[0][0], types[0][1]
    intro = f"对这组图像进行分析，检测到{T}种问题，共{N}处。"
    core = f"会影响发电。影响最严重的是{top_name}，{top['impact']}。"
    if T == 1: return intro + core
    others = "、".join([f"{v['cnt']}处{t}" for t,v in types[1:][:2]])
    tail = f"其余问题包括{others}{'等' if T>3 else ''}，亦会不同程度降低发电效率，建议按优先级依次处理。"
    return intro + core + tail

def TypeOne_id59(label):
    f = parse_label(label)
    if not f: return "对这组图像进行分析，未检测到异常情况，因此光伏组件发电不受影响。结论：不影响发电。"
    # 按缺陷类型聚合坐标（优先IR，无则VIS）
    g = {}
    for x in f:
        t = x['type']; c = x.get('ir_coords') or x.get('vis_coords')
        g.setdefault(t, []).append(c) if c else g.setdefault(t, [])
    join = lambda L: (L[0] if len(L)==1 else ("和".join(L) if len(L)==2 else "、".join(L[:-1])+"和"+L[-1]))
    T = len(g)

    # 只有一种问题
    if T == 1:
        (t, L) = next(iter(g.items()))
        loc = (f"出现在热成像{join(L)}的区域，" if L else "")
        return f"对这组图像进行分析，共检测到1种问题，为{t}，{loc}{descriptions.get(t,{}).get('impact','可能影响发电效率')}。结论：会影响发电，建议尽快处置以恢复发电水平。"

    # 多种问题（同类多处合并坐标）
    parts = []
    for t, L in g.items():
        loc = (f"出现在热成像{join(L)}的区域，" if L else "")
        parts.append(f"{t}，{loc}{descriptions.get(t,{}).get('impact','可能影响发电效率')}。")
    return "".join(parts) + generate_answer_impact_summary(label)

print("测试用例1 输出：")
print(TypeOne_id59(label1))
print("\n测试用例2 输出：")
print(TypeOne_id59(label2))
print("\n测试用例3 输出：")
print(TypeOne_id59(label3))
print("测试用例4 输出：")
print(TypeOne_id59(label4))

测试用例1 输出：
对这组图像进行分析，共检测到1种问题，为表面破损，出现在热成像坐标(365, 148)到(421, 176)的区域，因破损导致透光率下降，可能导致发电效率下降10%-30%，存在进水损坏电路的潜在风险。结论：会影响发电，建议尽快处置以恢复发电水平。

测试用例2 输出：
草木遮挡，出现在热成像坐标(73, 81)到(110, 108)的区域，会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率。边缘积尘，出现在热成像坐标(200, 300)到(250, 350)的区域，因积尘量少，污染程度较低，通常引起的发电效率下降小于10%。组串开路，出现在热成像坐标(557, 155)到(587, 165)的区域，会直接导致整个组串停止发电，形成大范围热斑，显著降低系统发电效率。对这组图像进行分析，检测到3种问题，共3处。会影响发电。影响最严重的是组串开路，会直接导致整个组串停止发电，形成大范围热斑，显著降低系统发电效率。其余问题包括1处草木遮挡、1处边缘积尘，亦会不同程度降低发电效率，建议按优先级依次处理。

测试用例3 输出：
表面破损，出现在热成像坐标(365, 148)到(421, 176)的区域，因破损导致透光率下降，可能导致发电效率下降10%-30%，存在进水损坏电路的潜在风险。草木遮挡，出现在热成像坐标(73, 81)到(110, 108)和坐标(200, 300)到(250, 350)的区域，会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率。边缘积尘，出现在热成像坐标(300, 400)到(350, 450)的区域，因积尘量少，污染程度较低，通常引起的发电效率下降小于10%。内部缺陷，出现在热成像坐标(557, 155)到(587, 165)的区域，属于隐性故障，会降低发电效率下降10%-50%，而且会持续恶化，存在局部瘫痪的潜在风险。对这组图像进行分析，检测到4种问题，共5处。会影响发电。影响最严重的是内部缺陷，属于隐性故障，会降低发电效率下降10%-50%，而且会持续恶化，存在局部瘫痪的潜在风险。其余问题包括1处表面破损、2处草木遮挡等，亦会不同程度降低发电效率，建议按优先级依次处理。
测试用例4 输出：
对

### TypeOne_id68
**存在缺陷的光伏组件的坐标位置是多少？**

In [ ]:
# 存在缺陷的光伏组件的坐标位置是多少？
def TypeOne_id68(label):
    faults = parse_label(label)
    if not faults:
        return "对这组图像进行分析，未检测到明显异常情况，因此无缺陷光伏组件的坐标位置。"
    intro = f"对这组图像进行分析，共检测到{len(faults)}处异常情况："
    details = []
    for i, fault in enumerate(faults, 1):
        fault_type = fault['type']
        ir_coords = fault['ir_coords']
        vis_coords = fault['vis_coords']
        coord_desc = []
        if ir_coords:
            coord_desc.append(f"在红外图像上位于{ir_coords}的范围")
        if vis_coords:
            coord_desc.append(f"可见光图像上位于{vis_coords}的范围")
        if coord_desc:
            details.append(f"异常{i}为{fault_type}，{'并且'.join(coord_desc)}。")
        else:
            details.append(f"异常{i}为{fault_type}，无可用坐标信息。")
    return intro + "".join(details) + "这些坐标可用于定位并检修受影响的组件。"

print("测试用例1 输出：")
print(TypeOne_id68(label1))
print("\n测试用例2 输出：")
print(TypeOne_id68(label2))
print("\n测试用例3 输出：")
print(TypeOne_id68(label3))
print("测试用例4 输出：")
print(TypeOne_id68(label4))

测试用例1 输出：
对这组图像进行分析，共检测到1处异常情况：异常1为表面破损，在红外图像上位于坐标(365, 148)到(421, 176)的范围并且可见光图像上位于坐标(1399, 617)到(1598, 725)的范围。这些坐标可用于定位并检修受影响的组件。

测试用例2 输出：
对这组图像进行分析，共检测到3处异常情况：异常1为草木遮挡，在红外图像上位于坐标(73, 81)到(110, 108)的范围并且可见光图像上位于坐标(259, 345)到(289, 398)的范围。异常2为边缘积尘，在红外图像上位于坐标(200, 300)到(250, 350)的范围并且可见光图像上位于坐标(800, 900)到(850, 950)的范围。异常3为组串开路，在红外图像上位于坐标(557, 155)到(587, 165)的范围。这些坐标可用于定位并检修受影响的组件。

测试用例3 输出：
对这组图像进行分析，共检测到5处异常情况：异常1为表面破损，在红外图像上位于坐标(365, 148)到(421, 176)的范围并且可见光图像上位于坐标(1399, 617)到(1598, 725)的范围。异常2为草木遮挡，在红外图像上位于坐标(73, 81)到(110, 108)的范围并且可见光图像上位于坐标(259, 345)到(289, 398)的范围。异常3为草木遮挡，在红外图像上位于坐标(200, 300)到(250, 350)的范围并且可见光图像上位于坐标(800, 900)到(850, 950)的范围。异常4为边缘积尘，在红外图像上位于坐标(300, 400)到(350, 450)的范围并且可见光图像上位于坐标(900, 1000)到(950, 1050)的范围。异常5为内部缺陷，在红外图像上位于坐标(557, 155)到(587, 165)的范围。这些坐标可用于定位并检修受影响的组件。
测试用例4 输出：
对这组图像进行分析，未检测到明显异常情况，因此无缺陷光伏组件的坐标位置。
